Housing Statistics for Scotland - Public sector vacant stock

In [1]:
from gssutils import *

scraper = Scraper('https://www2.gov.scot/Topics/Statistics/Browse/Housing-Regeneration/HSfS/VacantStock')
scraper

## Housing Statistics for Scotland - Public sector vacant stock

### Description

Information on local authority owned vacant stock is collected form each local
authority through the Housing Statistics Annual Return. Local authority
dwellings can be vacant for a variety of reasons, such as being part of a
planned disposal or modernisation/repair programme, or in low demand areas.

Figures on vacant dwellings are broken down by reason for vacancy, and length
of time the property has been vacant.



### Distributions

1. Local authority vacant stock by length of vacancy and reason for vacancy  ([MS Excel Spreadsheet](https://www2.gov.scot/Resource/0054/00540643.xls))


In [2]:
tabs = {tab.name: tab for tab in scraper.distribution().as_databaker()}
tabs.keys()

dict_keys(['DATA', 'NOTES', 'CONTENTS', 'tsAllVacant', 'tsTempHomeless', 'tsDemolish', 'tsModernise', 'tsLowDemand', 'tsOther', 'TblLASummary'])

In [3]:
tabs = scraper.distribution().as_databaker()

In [4]:
tenure = ['public-sector-vacant-stock',
          'public-sector-vacant-stock-used-as-temporary-accommodation-for-homeless',
          'public-sector-vacant-stock-awaiting-demolition',
          'public-sector-vacant-stock-forming-part-of-a-modernisation-scheme',
          'public-sector-vacant-stock-in-low-demand-areas',
          'public-sector-vacant-stock-other-types'
          ]

In [5]:
tidy = pd.DataFrame()
for i in range(3,9):
    print(i)
    tab = tabs[i]
    cell = tab.filter(contains_string('Public'))
    tl = cell.shift(0,1)
    area = tab.excel_ref('A1').expand(DOWN).is_not_blank().is_not_whitespace() -cell - tl
    year = tl.shift(0,1).fill(RIGHT).is_not_blank().is_not_whitespace()
    observations = area.fill(RIGHT).is_not_blank().is_not_whitespace()
    Dimensions = [
                HDim(year,'Year',DIRECTLY,ABOVE),
                HDim(tl,'Vacancy length',CLOSEST,ABOVE),
                HDim(area,'Area',DIRECTLY,LEFT),
                HDimConst('Measure Type', 'Count'),
                HDimConst('Unit', 'dwellings')
                ]
    c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)    
    table = c1.topandas()
    table['Tenure'] = tenure[i-3]
    tidy = pd.concat([tidy , table])  

3

4

5

6

7

8



In [6]:
import numpy as np
tidy['OBS'].replace('', np.nan, inplace=True)
tidy.dropna(subset=['OBS'], inplace=True)
if 'DATAMARKER' in tidy.columns:
    tidy.drop(columns=['DATAMARKER'], inplace=True)
tidy.rename(columns={'OBS': 'Value', 'Area':'Geography' }, inplace=True)
tidy['Value'] = tidy['Value'].astype(int)

In [7]:
tidy.rename(columns={'Year': 'Period'}, inplace=True)
tidy = tidy[tidy['Period'] != '']

In [8]:
tidy['Period'] = 'day/' + tidy['Period'].astype(str).str[:4] + '-03-31'

In [9]:
tidy['Vacancy length'] = tidy['Vacancy length'].map(
    lambda x: {
        'all vacant stock' : 'total', 'vacant for under 2 weeks' : 'under-2-weeks',
       'vacant for 2 to 6 weeks' : '2-to-6-weeks', 'vacant for 6 to 26 weeks' : '6-to-26-weeks',
       'vacant for 26 weeks to 2 years' : '26-weeks-to-2-years', 
        'vacant for longer than 2 years' : 'longer-than-2-years',
       'length of vacancy unknown' : 'unknown',
       'all vacant stock used as temporary accommodation for homeless' : 'total',
       'all vacant stock awaiting demolition' : 'total',
       'all vacant stock forming part of a modernisation scheme' :'total',
       'all vacant stock in low demand areas' : 'total',
       'all vacant stock of other types' : 'total'
        }.get(x, x))

In [10]:
tidy = tidy[tidy['Geography'] != 'Local Authorities']
tidy = tidy[tidy['Geography'] != 'Scottish Homes']

In [11]:
tidy['Geography'] = tidy['Geography'].map(
    lambda x: {
        'Scottish Borders, The' : 'Scottish Borders', 
        'Shetland' : 'Shetland Islands',
        'Na h-Eilanan Siar' : 'Na h-Eileanan Siar',
        'Orkney' : 'Orkney Islands'
        }.get(x, x))

In [12]:
scotland_gss_codes = pd.read_csv('scotland-gss.csv', index_col='Area')
tidy['Geography'] = tidy['Geography'].map(
    lambda x: scotland_gss_codes.loc[x]['Code']
)

In [13]:
tidy = tidy[['Period','Geography','Vacancy length','Tenure','Measure Type','Value','Unit']]

In [14]:
out = Path('out')
out.mkdir(exist_ok=True)
tidy.to_csv(out / 'observations.csv', index = False)

In [15]:
scraper.dataset.family = 'housing'
scraper.dataset.theme = THEME['housing-planning-local-services']
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())
    
schema = CSVWSchema('https://ons-opendata.github.io/ref_housing/')
schema.create(out / 'observations.csv', out / 'observations.csv-schema.json')

In [16]:
tidy

,Period,Geography,Vacancy length,Tenure,Measure Type,Value,Unit
0,day/1998-03-31,S92000003,total,public-sector-vacant-stock,Count,18117,dwellings
1,day/1999-03-31,S92000003,total,public-sector-vacant-stock,Count,19843,dwellings
2,day/2000-03-31,S92000003,total,public-sector-vacant-stock,Count,21530,dwellings
3,day/2001-03-31,S92000003,total,public-sector-vacant-stock,Count,22062,dwellings
4,day/2002-03-31,S92000003,total,public-sector-vacant-stock,Count,20746,dwellings
5,day/2003-03-31,S92000003,total,public-sector-vacant-stock,Count,16474,dwellings
6,day/2004-03-31,S92000003,total,public-sector-vacant-stock,Count,14399,dwellings
7,day/2005-03-31,S92000003,total,public-sector-vacant-stock,Count,14050,dwellings
8,day/2006-03-31,S92000003,total,public-sector-vacant-stock,Count,12060,dwellings
9,day/2007-03-31,S92000003,total,public-sector-vacant-stock,Count,10289,dwellings
